In [2]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
import os
from langgraph.types import Command
from langgraph.prebuilt import create_react_agent
from langchain.tools import tool
from typing_extensions import Literal
from langgraph.graph import MessagesState, StateGraph, START, END

In [3]:
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

llm = ChatOpenAI()
llm.invoke("Hey there, how are you?")

AIMessage(content="Hello! I'm just a computer program, so I don't have feelings, but I'm here and ready to help you with anything you need. How can I assist you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 37, 'prompt_tokens': 14, 'total_tokens': 51, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-Bo3rUGLM7Anq09wLNCX9z9ICfaIE0', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--ecfc1014-7ee9-455b-b187-e2f87ac49434-0', usage_metadata={'input_tokens': 14, 'output_tokens': 37, 'total_tokens': 51, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})